In [7]:
from torch import nn
import torch
from torch.utils.data import DataLoader
from DataModel import SegmentationData,base_image_path,base_imge_name,base_label_path,base_label_name,base_path

class Decoder(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super(Decoder, self).__init__()

        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv_relu = nn.Sequential(
            nn.Conv2d(middle_channels, out_channels, kernel_size=3,stride=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1),
            nn.ReLU(),
            )
  

    def forward(self, x1, x2):
        x1 = self.up(x1)
       
        x1 = torch.cat((x1, x2), dim=1)
        x1 = self.conv_relu(x1)
        return x1


    
    
class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels,use_maxpool=True):
        super(Encoder, self).__init__()
        
        layers=[]
        
        if use_maxpool==True:
            layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
        
        layers+=[
            nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=1),
            nn.ReLU(),
            nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1),
            nn.ReLU(),
        ]
        self.net=nn.Sequential(*layers)
        
    def forward(self,X):
        
        return self.net(X)

    
encoder_params=((3,64,False),(64,128),(128,256),(256,512),(512,1024))    
decoder_params=((1024,1024,512),(512,512,256),(256,256,128),(128,128,64)) 
class U_net(nn.Module):
    def __init__(self,encoder_params,decoder_params):
        super().__init__()
        maxpools=[]
        encoders=[]
        for param in encoder_params:
            encoders.append(Encoder(*param))
            maxpools.append(nn.MaxPool2d(kernel_size=2,stride=2))
            
        decoders=[]
        for param in decoder_params:
            decoders.append(Decoder(*param))
        
        self.encoders=nn.Sequential(*encoders)
        self.decoders=nn.Sequential(*decoders)
        self.finalconv=nn.Conv2d(64,21,kernel_size=1)
       
        
        
    def forward(self,X):

        x0=self.encoders[0](X)
        #print(x0.shape)
        x1=self.encoders[1](x0)
        #print(x1.shape)
        x2=self.encoders[2](x1)
        #print(x2.shape)
        x3=self.encoders[3](x2)
        #print(x3.shape)
        x4=self.encoders[4](x3)
        #print(x4.shape)
        e3=x3[:,:,4:60,4:60]
        x5=self.decoders[0](x4,e3)
        #print(x5.shape)
        e2=x2[:,:,16:120,16:120]
        x6=self.decoders[1](x5,e2)
        #print(x6.shape)
        e1=x1[:,:,40:240,40:240]
        x7=self.decoders[2](x6,e1)
        e0=x0[:,:,98:490,98:490]
        #print(x7.shape)
        x8=self.decoders[3](x7,e0)
        return self.finalconv(x8)





In [8]:
data=SegmentationData(base_path+'train.txt',base_image_path,base_label_path)

In [9]:
train_iter=DataLoader(data,batch_size=1)

In [10]:
net=U_net(encoder_params,decoder_params)

In [11]:
def precise(test_iter,net,device):
    total=0
    correct=0
    with torch.no_grad():
        for X,y in test_iter:
            X,y=X.to(device),y.to(device)
            y_hat=net(X)
            y_hat=torch.argmax(y_hat,dim=-1).flatten()
            y=y.flatten()
            ans=(y_hat==y)
            total+=len(y)
            correct+=ans.sum().item()
        
    print(correct)
    print(f"accuracy :{correct/total*100:>3f}% ")

    
def train(data_iter,entroy_iter,net,optimizer,lr_scheduler,loss_fn,epochs,device,epoch_data_num):
    matrix_x,matrix_loss,entroy_loss,entroy_x=[0],[0],[],[]
    total_loss=0
    batchs=len(data_iter)
    for epoch in range(epochs):
        now_num=0
        for X,y in data_iter:
 
            now_num+=len(X)
    
            net.train()
            
            X,y=X.to(device),y.to(device)
            optimizer.zero_grad()
            y_hat=net(X)
            y_hat=y_hat.permute(0,2,3,1).flatten(start_dim=0,end_dim=-2)
            y=y.flatten()
            loss=loss_fn(y_hat,y)
            loss.sum().backward()
            optimizer.step()
            total_loss+=loss.item()
            
            matrix_x.append(matrix_x[-1]+1)
           
            correct=(y==torch.argmax(y_hat,dim=-1)).sum().item()
            precise=correct*1.0/len(y)            
            matrix_loss.append(total_loss/(epoch*epoch_data_num+now_num))
            
            print(f"loss: {matrix_loss[-1]:>7f}   accuracy:{precise*100}%   now {matrix_x[-1]}/{batchs*epochs}" ,end='\r')
        
        lr_scheduler.step()
        torch.save(net.state_dict(), f"U-net.epoch{epoch+1}.bin")
    return net,matrix_x,matrix_loss,entroy_x,entroy_loss



from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
device='cuda' if torch.cuda.is_available() else 'cpu'
net=U_net(encoder_params,decoder_params).to(device)
optimizer=Adam(net.parameters(),lr=0.0006)
lr_scheduler=LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(2**epoch))
loss_fn=nn.CrossEntropyLoss()
'''for m in net.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_uniform_(m.weight)
'''



'for m in net.modules():\n    if isinstance(m, (nn.Conv2d, nn.Linear)):\n        nn.init.xavier_uniform_(m.weight)\n'

In [12]:
_,matrix_x,matrix_loss,entroy_x,entroy_loss=train(train_iter,train_iter,net,optimizer,lr_scheduler,loss_fn,4,device,len(data))



KeyboardInterrupt: 